## 一、初步使用RAG

### 1、关于RAG的几个问题

- 什么是文本嵌入？
- 文本拆分和编码？
- 使用向量数据库？
- 查询知识？

### 2、直接使用搜索结果

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_zhipu import ChatZhipuAI

## 本地搜索
search_db = lambda x: "阳台"

## 提示语模板
template = """
基于以下提供的信息回答问题:
{context}

Question: {question}

不要啰嗦。
"""

prompt = ChatPromptTemplate.from_template(template)

## 定义链
chain = (
    {
        "context": search_db,
        "question": RunnablePassthrough()
    }
    | prompt 
    | ChatZhipuAI() 
    | StrOutputParser()
)

chain.invoke("我们玩一个找猫的游戏，你的目标是找猫藏在家里哪个位置。你要努力去找哦！")

### 3、RAG的例子

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_zhipu import ChatZhipuAI, ZhipuAIEmbeddings

## 定义向量存储
db = Chroma.from_texts(
    [
        "我家有一条黄色的拉布拉多犬",
        "我家的狗狗喜欢吃蜂蜜",
        "我家的猫喜欢是白色的",
        "我家的绿色乌龟喜欢吃小鱼",
        "我家的金色小鱼喜欢吃蚯蚓",
        "我家的娃娃喜欢溜狗狗",
    ],
    embedding=ZhipuAIEmbeddings(),
)
retriever = db.as_retriever()

In [ ]:
db.similarity_search("你家的狗什么颜色?", k=3)

In [ ]:
db.similarity_search("你家的鱼什么颜色?", k=3)

In [ ]:
## 提示语模板
template = """
基于以下提供的信息回答问题:
{context}

Question: {question}

不要啰嗦。
"""

prompt = ChatPromptTemplate.from_template(template)

## 定义链
chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
    | prompt 
    | ChatZhipuAI() 
    | StrOutputParser()
)
chain.invoke("你猜猜我家有一只什么颜色的猫？")

In [ ]:
chain.invoke("你猜猜我家有一只什么颜色的Cat？")